In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def create_samples(n_clusters, n_samples_per_cluster, n_features, embiggen, seed):
    slices = []
    centroids = []
    temp_cent = ((0,0),(10,10),(-10,-10))
    for i in range(n_clusters):
        current_centroid = tf.constant(np.random.rand(1, n_features) * embiggen * 2 - embiggen,
                                       dtype=tf.float32, name="centroid_{}".format(i))
        samples = tf.random_normal((n_samples_per_cluster, n_features),
                                   mean=0.0, stddev=10.0, dtype=tf.float32, seed=seed, name="cluster_{}".format(i))


        #current_centroid = tf.constant(temp_cent[i], dtype=tf.float32, shape=(1,2))
        centroids.append(current_centroid)
        samples+=current_centroid
        slices.append(samples)
    samples = tf.concat(slices, 0, name='samples')
    centroids = tf.concat(centroids, 0, name="centroids")
    return centroids, samples

In [3]:
def plot_clusters(all_samples, centroids, n_samples_per_cluster):
    color = plt.cm.rainbow(np.linspace(0,1, len(centroids)))
    for i, centroid in enumerate(centroids):
        samples = all_samples[i*n_samples_per_cluster: (i+1)*n_samples_per_cluster]
        plt.scatter(samples[:,0], samples[:,1], c=color[i])
        plt.plot(centroid[0], centroid[1], markersize=35, marker="x", color='k', mew=10)
        plt.plot(centroid[0], centroid[1], markersize=35, marker="x", color='m', mew=5)
    plt.show()
    

In [4]:
# def choose_random_centroids(samples, n_clusters):
#     #n_samples = tf.shape(samples)[0]
#     n_samples = samples.shape.as_list()[0]
#     indexes = np.random.randint(0, n_samples, n_clusters)
#     return tf.gather(samples, indexes)

def choose_random_centroids(samples, n_clusters):
    n_features = tf.shape(samples)[1]
    return tf.random_crop(samples, (n_clusters,n_features))


In [5]:
def assign_to_nearest(samples, centroids):
    exp_vectors = tf.expand_dims(samples, 0)
    exp_centroids = tf.expand_dims(centroids, 1)
    distances = tf.reduce_sum((exp_vectors-exp_centroids)**2, 2)
    mins = tf.argmin(distances, 0)
    return mins
    

In [6]:
def compute_new_centroids(samples, nearest_indices, n_clusters):
    nearest_indices = tf.to_int32(nearest_indices)
    partitions = tf.dynamic_partition(samples, nearest_indices, n_clusters)
    new_centroids = tf.concat([tf.expand_dims(tf.reduce_mean(partition, 0),0) for partition in partitions], 0)
    return new_centroids

In [7]:
n_features = 2
n_clusters = 3
n_samples_per_cluster = 500
seed = 700
embiggen_factor = 40
np.random.seed(seed)

In [8]:
centroids, samples = create_samples(n_clusters, n_samples_per_cluster, n_features, embiggen_factor, seed)
initial_centroids = choose_random_centroids(samples, 3)
current_centroids = initial_centroids
#s = tf.InteractiveSession()

In [9]:
model = tf.global_variables_initializer()
for i in range(40):
        nearest_indices = assign_to_nearest(samples, current_centroids)
        current_centroids = compute_new_centroids(samples, nearest_indices, n_clusters)

In [ ]:
with tf.Session() as s:
    s.run(model)
    sample_values, initial_centroid_values, final_centroid_values = s.run([samples, initial_centroids,current_centroids])
    plot_clusters(sample_values, centroid_values, n_samples_per_cluster)
    plot_clusters(sample_values, final_centroid_values, n_samples_per_cluster)

In [10]:
with tf.Session() as s:
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("basic", s.graph)

In [ ]:

    

import tensorflow as tf
from tensorflow.python.client import timeline

x = tf.random_normal([5000, 5000], dtype=tf.float64)
y = tf.random_normal([5000, 5000], dtype=tf.float64)
res = tf.matmul(x, y)

# Run the graph with full trace option
sess = tf.Session(config=tf.ConfigProto(
  intra_op_parallelism_threads=10))

run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
run_metadata = tf.RunMetadata()
sess.run(res, options=run_options, run_metadata=run_metadata)

# Create the Timeline object, and write it to a json
tl = timeline.Timeline(run_metadata.step_stats)
ctf = tl.generate_chrome_trace_format()
with open('timeline.json', 'w') as f:
    f.write(ctf)
